In [1]:
import solopy
from solopy import _utils
import ccdproc
from astropy.io import fits
from astropy.time import Time
from pathlib import Path
import logging
import bz2
import shutil # 임시 디렉터리 삭제용

In [2]:
subdir = "2025_0717" # observation date

In [3]:
# Data directory
DATADIR = Path("../solo-data")

# Lv0 data directory
LV0DIR = DATADIR/"Lv0"
LV0_subdir = LV0DIR / subdir

# Lv1 data directory
LV1DIR = DATADIR/"Lv1"
LV1_subdir = LV1DIR / subdir
LV1_subdir.mkdir(parents=True, exist_ok=True)

# Master calibration files directory
MASTERDIR = DATADIR / "calibration_files"
MASTERDIR.mkdir(parents=True, exist_ok=True)

# Log directory
LOGDIR = DATADIR/"log"
LOGDIR.mkdir(parents=True, exist_ok=True)

fpath_log_lv0 = LOGDIR/f'lv0_{subdir}.log' # log file path for Lv0
if fpath_log_lv0.exists():
    fpath_log_lv0.unlink() # remove existing log file
    
fpath_log_lv1 = LOGDIR/f'lv1_{subdir}.log' # log file path for Lv1
if fpath_log_lv1.exists():
    fpath_log_lv1.unlink() # remove existing log file

fpath_log_comb = LOGDIR/f'combmaster_{subdir}.log' # log file path for CombMaster
if fpath_log_comb.exists():
    fpath_log_comb.unlink() # remove existing log file

In [4]:
# Logger
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[
                        logging.StreamHandler(),
                        logging.FileHandler(LOGDIR/"solo-example.log")
                    ])
logger = logging.getLogger(__name__)

In [5]:
# Step 1: Batch Decompress
logger.info("--- Step 1: Batch Decompress ---")
_utils.batch_decompress(LV0_subdir, LV0_subdir, delete_source=True)

2025-10-27 20:23:33,802 [INFO] --- Step 1: Batch Decompress ---
2025-10-27 20:23:33,803 [INFO] Decompressing 147 files from ../solo-data/Lv0/2025_0717 to ../solo-data/Lv0/2025_0717...
Decompressing: 100%|██████████| 147/147 [01:22<00:00,  1.78it/s]


In [6]:
# Step 2: Lv0 Header Update
logger.info("--- Step 2: Lv0 Header Update ---")
lv0 = solopy.Lv0(log_file=fpath_log_lv0)

fpaths_fits = list(LV0_subdir.glob("*.fits"))
if not fpaths_fits:
    raise FileNotFoundError(f"No .fits files found in {LV0_subdir}.")

for fpath in fpaths_fits:
    lv0.update_header(fpath) # Update header in-place

2025-10-27 20:25:46,881 [INFO] --- Step 2: Lv0 Header Update ---
2025-10-27 20:25:46,953 [INFO] Updated LV0 header: dk03_007.fits
2025-10-27 20:25:46,994 [INFO] Updated LV0 header: dk02_007.fits
2025-10-27 20:25:47,037 [INFO] Updated LV0 header: dark008_002.fits
2025-10-27 20:25:47,082 [INFO] Updated LV0 header: bias_006.fits
2025-10-27 20:25:47,127 [INFO] Updated LV0 header: ELENORA_031_20250717061504.fits
2025-10-27 20:25:47,170 [INFO] Updated LV0 header: ELENORA_032_20250717061637.fits
2025-10-27 20:25:47,212 [INFO] Updated LV0 header: CARLOVA_001_20250717090310.fits
2025-10-27 20:25:47,254 [INFO] Updated LV0 header: dark004_009.fits
2025-10-27 20:25:47,295 [INFO] Updated LV0 header: dark090_002.fits
2025-10-27 20:25:47,336 [INFO] Updated LV0 header: dk01_002.fits
2025-10-27 20:25:47,377 [INFO] Updated LV0 header: CARLOVA_012_20250717092002.fits
2025-10-27 20:25:47,419 [INFO] Updated LV0 header: ELENORA_014_20250717054859.fits
2025-10-27 20:25:47,462 [INFO] Updated LV0 header: CARLO

In [7]:
logger.info("--- Step 3: CombMaster Frame Generation ---")
comb = solopy.CombMaster(log_file=fpath_log_comb)

# Collect all Lv0 fits files
all_lv0_fits = ccdproc.ImageFileCollection(LV0_subdir, glob_exclude="*.bz2", glob_include="*.fits")

# Master bias
bias_files = all_lv0_fits.files_filtered(IMAGETYP="BIAS", include_path=True)
if bias_files:
    comb.make_mbias(bias_files, MASTERDIR)
    
# Master dark
dark_files = all_lv0_fits.files_filtered(IMAGETYP="DARK", include_path=True)
if dark_files:
    comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')

2025-10-27 20:25:57,412 [INFO] --- Step 3: CombMaster Frame Generation ---
2025-10-27 20:25:57,621 [INFO] Starting master bias combination...
2025-10-27 20:25:57,622 [INFO] Loading 8 frames...
2025-10-27 20:25:57,640 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,658 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,674 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,691 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,711 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,732 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,752 [INFO] using the unit adu passed to the FITS reader instead of the unit adu

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 10 chunks to limit memor

2025-10-27 20:26:08,390 [INFO] Master bias saved to kl4040.bias.comb.20250717.fits
2025-10-27 20:26:08,509 [INFO] Starting master dark creation...
2025-10-27 20:26:08,527 [INFO] Loading 81 frames...
2025-10-27 20:26:08,545 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,562 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,580 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,639 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,666 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,682 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,699 [INFO] using the unit adu passed to the FITS reader instead of the un

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-10-27 20:26:08,752 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,770 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,786 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,803 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,821 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,839 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,857 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,874 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,892 [INFO] using the unit adu passed

...

2025-10-27 20:27:43,892 [INFO] Master dark saved to kl4040.dark.2s.comb.20250717.fits
2025-10-27 20:27:43,892 [INFO] Combining 9 darks for exptime 3.0s...
2025-10-27 20:27:43,937 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 20:27:56,534 [INFO] Master dark saved to kl4040.dark.3s.comb.20250717.fits
2025-10-27 20:27:56,534 [INFO] Combining 9 darks for exptime 5.0s...
2025-10-27 20:27:56,581 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 20:28:09,332 [INFO] Master dark saved to kl4040.dark.5s.comb.20250717.fits
